In [1]:
# Jannat Butt
# 21I-0259
# AI-K

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import accuracy_score
import math

In [3]:
df=pd.read_csv("play_tennis.csv")

In [4]:
df

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [5]:

class DecisionTree:
    def __init__(self):
        self.tree = None

    def dataset_entropy(self, df, col):
        targets = df[col].unique()
        No = df[col].value_counts().get(targets[0], 0)
        Yes = df[col].value_counts().get(targets[1], 0)
        overall_entropy = -(Yes / len(df)) * (math.log2(Yes / len(df))) - (No / len(df)) * (math.log2(No / len(df)))
        return overall_entropy

    def find_entropy(self, overall_entropy, col, df):
        unique_values = df[col].unique()
        uniques = {i: df[df[col] == i].shape[0] for i in unique_values}
        #print(uniques)
        uniques_yes_no = {}

        for key in uniques:
            new_key_yes = key + "_Yes"
            uniques_yes_no[new_key_yes] = len(df[(df[col] == key) & (df["play"] == "Yes")])

        for key in uniques:
            new_key_yes = key + "_No"
            uniques_yes_no[new_key_yes] = len(df[(df[col] == key) & (df["play"] == "No")])

        grouped_counts = defaultdict(lambda: defaultdict(int))
        entropy = {}

        for key, count in uniques_yes_no.items():
            prefix, stripped_key = key.split('_', 1)
            grouped_counts[prefix][stripped_key] += count

        for key in grouped_counts:
            if grouped_counts[key]["Yes"] == 0:
                entropy[key] = (grouped_counts[key]["No"] / uniques[key]) * (
                        math.log2(grouped_counts[key]["No"] / uniques[key]))
            elif grouped_counts[key]["No"] == 0:
                entropy[key] = (grouped_counts[key]["Yes"] / uniques[key]) * (
                        math.log2(grouped_counts[key]["Yes"] / uniques[key]))
            else:
                entropy[key] = -(
                        (grouped_counts[key]["Yes"] / uniques[key]) * (
                        math.log2(grouped_counts[key]["Yes"] / uniques[key]))) - \
                               ((grouped_counts[key]["No"] / uniques[key]) * (
                                       math.log2(grouped_counts[key]["No"] / uniques[key])))

        result = 0
        for key in uniques:
            result += (uniques[key] / len(df)) * entropy[key]

        IG = overall_entropy - result
        return IG
    
    def predict(self,input_data, decision_tree):
 
        current_node = decision_tree

        while isinstance(current_node, dict):
            attribute = current_node['attribute']
            #print(attribute,"\n")
            attribute_value = input_data[attribute]
            #print( input_data[attribute])

            # Follow the branch based on the attribute value
            current_node = current_node['branches'][attribute_value]

        return current_node

    def accuracy_find(self,predictions):
        y_new=[0 if label=="No" else 1 for label in predictions]
        actual=[0 if label=="No" else 1 for label in df["play"]]
        accuracy=accuracy_score(y_new,actual)
        return accuracy
        
    def make_predictions(self,df,decision_tree):
        predictions=[]
        for i in df.iloc[:,1:-1].values:
            test_diction=dict(zip(df.columns[1:-1],i))
            predictions.append(self.predict(test_diction, decision_tree))
            
        return predictions
            
    def build_decision_tree(self, df):
        IG = dict()
        overall_entropy = self.dataset_entropy(df, "play")

        for i in df.columns[1:-1]:
            IG[i] = self.find_entropy(overall_entropy, i, df)

        root = max(IG, key=IG.get)

        tree = dict()
        tree['attribute'] = root
        tree["branches"] = {}

        for i in df[root].unique():
            subset = df[df[root] == i]
            unique_targets = subset['play'].nunique()

            if unique_targets == 1:
                tree["branches"][i] = subset['play'].iloc[0]
            else:
                tree["branches"][i] = self.build_decision_tree(subset)

        return tree
    
     

dt = DecisionTree()
decision_tree = dt.build_decision_tree(df)
predictions=dt.make_predictions(df,decision_tree)
accuracy=dt.accuracy_find(predictions)
print("accuracy:",accuracy)

accuracy: 1.0
